## Check system info (CoLab used)

In [ ]:
print("CPU Status:")
!cat /proc/cpuinfo | grep model\ name 
print("\nDisk Status:")
!df -lh 
print("\nRAM Status:")
!free -h 

## Set GoogleDrivePath (CoLab used)

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
GDrivePath = os.path.join(os.getcwd(),'drive','My Drive')
if os.path.exists(GDrivePath): 
   import sys
   sys.path.append(GDrivePath)
   print('GoogleDrive Ok')  
else:
   print ('GoogleDrive Failure') 

## Import 

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt 
from datetime import datetime
from ReadTFRecords import InputDataPipeline as IDP 
from ModelTools import ConfusionMatrixAnalysis as CMA
from ModelTools import PlotConfusionMatrix, FigToSummary
print("\nGPU Status:")
print (tf.test.gpu_device_name()) 

## Build some  tools

In [ ]:
def PlotAccLoss(result,save=False):
    steps_per_epoch = data_size['train']/batch_size['train'] 
    epochs = [s/steps_per_epoch for s in result['step']] 

    fig1, ax1 = plt.subplots(num='Acc')
    ax1.set_title('Accuracy',fontdict={'fontsize':20,'position':(0.5, 0.08)})
    ax1.set_ylabel('Accuracy') 
    ax1.set_xlabel('Steps')    
    ax1.plot(result['step'],result['train_acc'],'r-', label='train_acc')
    ax1.plot(result['step'],result['vali_acc'],'b-', label='vali_acc')
    ax1.tick_params(axis='x', labelcolor='k',direction='in')   
    ax2 = ax1.twiny() # share same y axis
    ax2.set_xlabel('Epochs',color='g')
    ax2.plot(epochs,result['train_acc'],'r-', label='train_acc')
    ax2.plot(epochs,result['vali_acc'],'b-', label='vali_acc')    
    ax2.tick_params(axis='x', labelcolor='g',direction='in')
    ax1.grid(which="both", visible=False)
    ax2.grid(which='both', visible=False)    
    plt.ylim(0,1.1)
    plt.legend(loc='best')
    fig1.tight_layout()
    if save: plt.savefig(os.path.join(save_main_path,'Acc.png'),bbox_inches='tight',dpi=150)     
    plt.show()

    fig3, ax3 = plt.subplots(num='Loss')
    ax3.set_title('Loss',fontdict={'fontsize':20,'position':(0.5, 0.80)})
    ax3.set_ylabel('Loss')
    ax3.set_xlabel('Steps')    
    ax3.plot(result['step'],result['train_loss'],'r-', label='train_loss')
    ax3.plot(result['step'],result['vali_loss'],'b-', label='vali_loss')
    ax3.tick_params(axis='x', labelcolor='k',direction='in')   
    ax4 = ax3.twiny() 
    ax4.set_xlabel('Epochs',color='g')
    ax4.plot(epochs,result['train_loss'],'r-', label='train_loss')
    ax4.plot(epochs,result['vali_loss'],'b-', label='vali_loss')    
    ax4.tick_params(axis='x', labelcolor='g',direction='in') 
    ax3.grid(which="both", visible=False)
    ax4.grid(which='both', visible=False)
    plt.legend(loc='best')
    fig1.tight_layout()
    if save: plt.savefig(os.path.join(save_main_path,'Loss.png'),bbox_inches='tight',dpi=150)      
    plt.show()
      
def SaveAccLossValue(result):
    my_data_result = pd.DataFrame({
                      "step": result['step'],
                      "train_acc": result['train_acc'],  
                      "vali_acc": result['vali_acc'],
                      "train_loss": result['train_loss'],  
                      "vali_loss": result['vali_loss'],                       
                      })         
    my_data_result.to_csv(os.path.join(save_main_path,'AccLossValue.csv'),sep=',') 
    
def EpochStepConverter(numbers, unit, batch_size, data_size):
    if unit=='step': # numbers*epochs_per_step
       steps = numbers 
       epochs = numbers*(batch_size/data_size)        
    elif unit=='epoch': # numbers*steps_per_epoch # step is element unit and  must be int
       steps = numbers*round(data_size/batch_size)
       epochs = numbers
    else:
       print ("unit must be step or epoch") 
       return 0, 0 
    return steps, epochs  

## Build 1D-CNN architectures

In [ ]:
from CnnModelClass import Classifier # or import ClassifierM4 directly if don't need to rewrite model
class ClassifierM4(Classifier): #1-D CNN
    
    def __init__(self, sess, **kwargs):        
        self.sess = sess
    
    def input_layer(self, name="InputData"): 
        
        def img_profile(shape, pix_area): #[15,480,3]            
            # luma(BT.709)=R*0.2126 + G*0.7152 + B*0.0722
            # luma(BT.601)=R*0.299 + G*0.587 + B*0.114  
            ysize, xsize = shape
            #pix_gray = pix_area[:,:,:,0]*0.2126 + pix_area[:,:,:,1]*0.7152 + pix_area[:,:,:,2]*0.0722
            pix_gray = pix_area[:,:,:,0]*0.333 + pix_area[:,:,:,1]*0.333 + pix_area[:,:,:,2]*0.333          
            #print (pix_gray) #(?, 15, 480)
            intensity = tf.reduce_sum(pix_gray, 1)/ysize 
            #print (intensity) #(?, 480)
            return intensity   
 
        with tf.name_scope(name):
          self.x_img = tf.placeholder(tf.float32, [None, self.image_shape[0], self.image_shape[1], 3], name='x_img')     
          self.x_img_profile = tf.reshape(img_profile(self.image_shape, self.x_img),[-1, self.image_shape[1] , 1], name="img2profile") # NWC formet                                         
          self.y_label = tf.placeholder(tf.uint8, [None, self.label_size],name='y_label')  
                                        
    def optimizer(self, learn_rate, true_label, pred_label, loss_name="Loss"): 
        with tf.name_scope(loss_name):
          self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=true_label, logits=pred_label))        
        with tf.name_scope('Trainning_step'):
          self.update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS) 
          with tf.control_dependencies(self.update_ops):
            #learn_rate = tf.cond(tf.less(self.loss, tf.constant(1.175)),lambda: 0.000000001,lambda: learn_rate)                                                 
            self.train_step = tf.train.AdamOptimizer(learn_rate).minimize(self.loss)                                                                                           
                                                 
    def hidden_layers(self):                                        
        self.conv1d_layer(input_neuro=self.x_img_profile, 
                          kernel_para=[5,1,self.convolution_kernel_size],
                          name="Convolution1D_layer_1")
        self.conv1d_layer(input_neuro=self.conv, 
                          kernel_para=[15,self.convolution_kernel_size,self.convolution_kernel_size*2],
                          name="Convolution1D_layer_2")
        self.conv1d_layer(input_neuro=self.conv, 
                          kernel_para=[45,self.convolution_kernel_size*2,self.convolution_kernel_size*2],
                          name="Convolution1D_layer_3")                                                 
        self.conv1d_layer(input_neuro=self.conv, 
                          kernel_para=[90,self.convolution_kernel_size*2,self.convolution_kernel_size*3],
                          name="Convolution1D_layer_4")
        self.conv1d_layer(input_neuro=self.conv, 
                          kernel_para=[105,self.convolution_kernel_size*3,self.convolution_kernel_size*3],
                          name="Convolution1D_layer_5")            
        self.pooling_layer(input_neuro=self.conv, 
                           name="Pooling_2x1", avg=False, one_d=True)                                                 
        self.flatten(input_neuro=self.pool, 
                           name="Flatten")  
        self.fully_connected_layer(input_neuro=self.flatten ,
                                   input_neuro_size=self.flatten_size,
                                   output_neuro_size=self.fc_layer_neuro_size, 
                                   name="FullyConnected_Layer_1")           
        self.fully_connected_layer(input_neuro=self.fc ,
                                   input_neuro_size=self.fc_layer_neuro_size,
                                   output_neuro_size=int(self.fc_layer_neuro_size/2), 
                                   name="FullyConnected_Layer_2")

## Build training process

In [ ]:
def RunTestSet(model, data, save=True, savepath = ''):     
    image_batch, label_batch = data.feeder(data.test_iterator)                
    test_accuracy, confusion_mtx = model.sess.run([model.accuracy,model.confusion],
                                             feed_dict={model.x_img: image_batch*(1./255), 
                                                        model.y_label: label_batch,
                                                        model.keep_prob: 1.0})
    print ("Test set Acc (Latest model) : %4.2f%%"%(test_accuracy*100))
    
    if save: 
       savepath = os.path.join(save_main_path,'TestSetCM_LatestModel.png')
       txt_savepath = os.path.join(save_main_path,'TestSetCM_LatestModel_Acc%4.2f.csv'%(test_accuracy*100))
       np.savetxt(txt_savepath, confusion_mtx, delimiter=',', fmt=str("%d"))
                                   
    cm_fig = PlotConfusionMatrix(confusion_mtx, show=True, savepath=savepath, colab=True)
    cm_analysis = CMA(confusion_mtx)            
    print (cm_analysis['report'])
                                   
    return confusion_mtx, cm_analysis

In [ ]:
def RunTraining(iter_number, earlystop=False, evalValiConfusion=False):  
    
    sess = tf.Session()            
    data = IDP(sess,inputdata_path,batch_size,image_shape,label_size)    
    model = ClassifierM4(sess) 
    model.init_model(image_shape, label_size, convolution_kernel_size,
                     fc_layer_neuro_size, learn_rate, save_main_path)
                          
    # Set early_stop param (vali_loss based)
    steps_per_epoch = round(data_size['train']/batch_size['train'])
    #no_improvement_steps = 10*steps_per_epoch 
    no_improvement_steps = int(iter_number/3)
    last_improvement_step = 0    
    best_vali_loss = 99     
    
    # Init list of recording acc loss value (plot used) 
    record_steps = []
    train_acc_val = []        
    vali_acc_val = []   
    train_loss_val = []        
    vali_loss_val = []
    
    current_epoch = 0
    steps = range(1,iter_number+1)
    StartTime = datetime.now()      
    for step in steps:        
        # Optimize 
        image_batch, label_batch = data.feeder(data.train_iterator)
        model.sess.run(model.train_step, 
                       feed_dict={model.x_img: image_batch*(1./255), 
                                  model.y_label: label_batch, 
                                  model.keep_prob: dropout,
                                  model.training: True}) 
        
        print ("\r"+"Epoch %4d : %3d/%3d "%(current_epoch+1, step-current_epoch*steps_per_epoch, steps_per_epoch), end="\r")
        
        # Record acc and loss value of each 10 steps or 1 epoch 
        if step % 10 == 0 or step % (1*steps_per_epoch) == 0 or step == steps[-1]:
           summary_train, train_loss, train_accuracy = model.sess.run([model.summary_merged_train, model.loss, model.accuracy],
                                              feed_dict={model.x_img: image_batch*(1./255), 
                                                         model.y_label: label_batch,
                                                         model.keep_prob: 1.0})
           image_batch, label_batch = data.feeder(data.vali_iterator)     
           summary_vali, vali_loss, vali_accuracy = model.sess.run([model.summary_merged_vali, model.loss, model.accuracy],
                                              feed_dict={model.x_img: image_batch*(1./255), 
                                                         model.y_label: label_batch,
                                                         model.keep_prob: 1.0}) 
           # Record acc loss value
           record_steps.append(step)
           train_acc_val.append(train_accuracy)       
           vali_acc_val.append(vali_accuracy)           
           train_loss_val.append(train_loss)       
           vali_loss_val.append(vali_loss)                 
           model.train_writer.add_summary(summary_train, step) 
           model.train_writer.add_summary(summary_vali, step)                   
           ##show_info = 'Step: %d  Train_Acc: %4.2f%%  Vali_Acc: %4.2f%%  Train_Loss: %1.4f  Vali_Loss: %1.4f'%(step,train_accuracy*100,vali_accuracy*100,train_loss,vali_loss)       

           # For each epoch
           if step % (1*steps_per_epoch) == 0:
              current_epoch += 1 
              ##show_info = ">>> Epoch: %d  Vali_Acc: %4.2f%%  Vali_Loss: %1.4f"%(current_epoch, vali_accuracy*100, vali_loss)                       
              show_info = "\n"+"Train_Acc: %4.2f%%  Vali_Acc: %4.2f%%  Train_Loss: %1.4f  Vali_Loss: %1.4f "%(train_accuracy*100,vali_accuracy*100,train_loss,vali_loss )              
              if evalValiConfusion:   
                 confusion_mtx = model.sess.run(model.confusion,
                                             feed_dict={model.x_img: image_batch*(1./255), 
                                                        model.y_label: label_batch,
                                                        model.keep_prob: 1.0})
                 cm_fig = PlotConfusionMatrix(confusion_mtx, show=False, colab=True)
                 cm_summery = FigToSummary(cm_fig,tag='ValiSet/ConfusionMatrix')
                 model.train_writer.add_summary(cm_summery, current_epoch)
                 cm_analysis = CMA(confusion_mtx)          
                 weighted_avg = cm_analysis['weighted_avg'] # avg of precision, recall, f1 
                 ##show_info = ">>> Epoch: %d  Vali_Acc: %4.2f%%  Vali_Loss: %1.4f  Precision: %4.2f%%  Recall: %4.2f%%"%(current_epoch, vali_accuracy*100, vali_loss, weighted_avg[0]*100,weighted_avg[1]*100) 
                 show_info = "\n"+"Train_Acc: %4.2f%%  Vali_Acc: %4.2f%%  Train_Loss: %1.4f  Vali_Loss: %1.4f  Precision: %4.2f%%  Recall: %4.2f%%"%(train_accuracy*100,vali_accuracy*100,train_loss,vali_loss,weighted_avg[0]*100,weighted_avg[1]*100)            
              if vali_loss < best_vali_loss: 
                 best_vali_loss = vali_loss                 
                 last_improvement_step = step # or epoch
                 if earlystop: # (set false in colab)  
                    model.saver_earlystop.save(model.sess, model_earlystop_savepath)
              print (show_info)     
           ##print (show_info)         

        # EarlyStopping
        if earlystop and step-last_improvement_step >= no_improvement_steps:
           print("-------------------------------------------------------")
           print("No improvement found in a while, stopping optimization.")
           print("Stop at step : %d"%(step))        
           break      
        
    EndTime = datetime.now()
    print("-------------------------------------------------------")
    print("Training completed!")
    print('Time usage : %s '%str(EndTime-StartTime))     
    print("Final improvement step : %d  Vali_Loss %1.4f"%(last_improvement_step,best_vali_loss))
    model.train_writer.add_graph(model.sess.graph)                                      
    model.train_writer.close() 
    model.saver.save(model.sess, model_savepath)        
    print("-------------------------------------------------------")   
    confusion_mtx, cm_analysis = RunTestSet(model, data, save=True)      
    model.sess.close()         
    print("-------------------------------------------------------")
    result = {'step':record_steps,
              'train_acc':train_acc_val,
              'vali_acc':vali_acc_val,
              'train_loss':train_loss_val,
              'vali_loss':vali_loss_val,
              'test_cm':confusion_mtx,
              'test_cm_report':cm_analysis,
              'last_improvement_step':last_improvement_step}        
    return result  


## Set training parameters (Local used)

In [ ]:
GDrivePath = os.path.join(os.getcwd())
# Set static parameter 
input_main_path = os.path.join(GDrivePath,'InputData') 
save_main_path = os.path.join(GDrivePath,'OutputModel',datetime.now().strftime("%Y%m%d%H%M"))
inputdata_path = {'train':os.path.join(input_main_path,'MyDataV4_train.tfrecords'),
                  'vali':os.path.join(input_main_path,'MyDataV4_vali.tfrecords'),
                  'test':os.path.join(input_main_path,'MyDataV4_test.tfrecords')}                
model_savepath = os.path.join(save_main_path,'MyDataV4_m4')
model_earlystop_savepath = os.path.join(save_main_path,'MyDataV4_m4_earlystop')    
if not os.path.exists(save_main_path): os.makedirs(save_main_path)
from ReadTFRecords import counter_TFRecord_datasize as ctd
data_size = {'train':ctd(inputdata_path['train']),'vali':ctd(inputdata_path['vali']),'test':ctd(inputdata_path['test'])}                                         
label_size = 7
image_shape = [15,480]
        
# Set hyper parameter 
batch_size = {'train':125,'vali':data_size['vali'],'test':data_size['test']}
learn_rate = 0.01 #0.0001 
fc_layer_neuro_size = 1280 #  
convolution_kernel_size = 32 #
dropout = 0.7

# Set steps or epochs
steps, epochs = EpochStepConverter(45,'epoch',batch_size['train'],data_size['train'])
#steps, epochs = EpochStepConverter(110,'step',batch_size['train'],data_size['train'])
print ("Iter steps = %d (%.2f epochs)"%(steps,epochs))

## Start training (Local used)

In [ ]:
earlystop = True
tf.reset_default_graph()
result = RunTraining(steps, earlystop=earlystop, evalValiConfusion=True)
PlotAccLoss(result,save=True)
SaveAccLossValue(result)
if earlystop and result['last_improvement_step'] != steps :
   from QuickFunctions import PredTestSet
   PredTestSet(model_earlystop_savepath, inputdata_path['test'], colab=True)

## Set training parameters (CoLab used)

In [ ]:
# Set static parameter 
input_main_path = os.path.join(GDrivePath,'InputData') 
save_main_path = os.path.join(GDrivePath,'OutputModel',datetime.now().strftime("%Y%m%d%H%M"))
inputdata_path = {'train':os.path.join(input_main_path,'MyDataV4_train.tfrecords'),
                  'vali':os.path.join(input_main_path,'MyDataV4_vali.tfrecords'),
                  'test':os.path.join(input_main_path,'MyDataV4_test.tfrecords')}                
model_savepath = os.path.join(save_main_path,'MyDataV4_m4')
#model_earlystop_savepath = os.path.join(save_main_path,'MyDataV1_m2_earlystop')    
if not os.path.exists(save_main_path): os.makedirs(save_main_path)
from ReadTFRecords import counter_TFRecord_datasize as ctd
data_size = {'train':ctd(inputdata_path['train']),'vali':ctd(inputdata_path['vali']),'test':ctd(inputdata_path['test'])}                                         
label_size = 7
image_shape = [15,480]
        
# Set hyper parameter 
batch_size = {'train':200,'vali':data_size['vali'],'test':data_size['test']}
learn_rate = 0.01 #0.0001 
fc_layer_neuro_size = 1280 #  
convolution_kernel_size = 32 #
dropout = 0.6

# Set steps or epochs
steps, epochs = EpochStepConverter(200,'epoch',batch_size['train'],data_size['train'])
#steps, epochs = EpochStepConverter(700,'step',batch_size['train'],data_size['train'])
print ("Iter steps = %d (%.2f epochs)"%(steps,epochs))

## Start training (CoLab used)

In [ ]:
earlystop = False
tf.reset_default_graph()
result = RunTraining(steps, earlystop=earlystop, evalValiConfusion=True)
PlotAccLoss(result,save=True)
SaveAccLossValue(result)

## Pred. MyRealImgs (Reload latest model)

In [ ]:
#import importlib
#import QuickFunctions 
#importlib.reload(QuickFunctions)
from QuickFunctions import PredMyRealImgs
#model_path = os.path.join(GDrivePath,'OutputModel', '201902100859','MyDataV2_m4') 
model_path = model_savepath 
imgfolder = os.path.join(GDrivePath,'RawData','MyRealImgs') 
PredMyRealImgs(model_path, imgfolder=imgfolder,colab=True) 


## Pred. MyRealImgs (Reload earlystoping model)

In [ ]:
from QuickFunctions import PredMyRealImgs
model_path = model_earlystop_savepath 
imgfolder = os.path.join(GDrivePath,'RawData','MyRealImgs') 
PredMyRealImgs(model_path, imgfolder=imgfolder,colab=True) 

## Model to .pb file

In [ ]:
from s5_DeployPbFile import ModelToPB
#model_path = os.path.join(GDrivePath,'OutputModel', '201902100859','MyDataV2_m4')
model_path = model_savepath # model_earlystop_savepath 
pbfile_path = model_path+".pb"
ModelToPB(model_path,pbfile_path)

## Save .ipynb by ctrl+s then Run the cell to copy file to save_main_path

In [ ]:
from shutil import copy
copy(os.path.join(GDrivePath,"colab_used.ipynb"),
     os.path.join(save_main_path,"Training.ipynb"))

## Pred. MyRealImgs (select other model)

In [ ]:
from QuickFunctions import PredMyRealImgs
model_path = os.path.join(GDrivePath,'OutputModel', '201902082017','MyDataV1_m4')
#model_path = model_path+".pb"
imgfolder = os.path.join(GDrivePath,'RawData','MyRealImgs') 
PredMyRealImgs(model_path, imgfolder=imgfolder,colab=True) 